In [16]:
import os 
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [17]:
import torch
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# Own library imports
from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My
from vecopsciml.kernels.derivative import DerivativeKernels

# Function from this project
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

# Import model
from architectures.pgnniv_baseline import PGNNIVBaseline

In [29]:
# Dataset
dataset = 'non_linear'
N_data = 100
noise = 1

data_name = dataset + '_' + str(N_data) + '_' + str(noise)
print(data_name)

non_linear_100_1


In [30]:
# Model
model = 'baseline'
n_modes = 10

model_name = model + '_model_' + str(n_modes)
print(model_name)

baseline_model_10


In [20]:
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../"))
DATA_PATH = os.path.join(ROOT_PATH, r'data/', data_name, data_name) + '.pkl'
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/', data_name)
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/', data_name, model_name)

In [21]:
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear_100_1
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear_100_1/baseline_model_10


In [22]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear_100_1/non_linear_100_1.pkl


In [23]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [24]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [25]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [26]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, n_modes, 10, 20]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [27]:
# Load model and the optimizer
model = PGNNIVBaseline(input_shape, predictive_layers, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 100

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 9.803e+08, Test loss: 9.784e+08, MSE(e): 9.783e+01, MSE(pi1): 1.919e+02, MSE(pi2): 4.049e+01, MSE(pi3): 6.130e-01
Epoch 10, Train loss: 7.897e+07, Test loss: 7.776e+07, MSE(e): 7.876e+00, MSE(pi1): 1.851e+01, MSE(pi2): 4.094e+00, MSE(pi3): 2.099e-01
Epoch 20, Train loss: 5.315e+07, Test loss: 5.669e+07, MSE(e): 5.310e+00, MSE(pi1): 2.603e+00, MSE(pi2): 2.267e+00, MSE(pi3): 2.121e-01
Epoch 30, Train loss: 3.906e+07, Test loss: 4.766e+07, MSE(e): 3.901e+00, MSE(pi1): 2.665e+00, MSE(pi2): 1.779e+00, MSE(pi3): 2.110e-01
Epoch 40, Train loss: 3.264e+07, Test loss: 3.810e+07, MSE(e): 3.259e+00, MSE(pi1): 2.604e+00, MSE(pi2): 1.481e+00, MSE(pi3): 2.134e-01
Epoch 50, Train loss: 3.102e+07, Test loss: 3.788e+07, MSE(e): 3.098e+00, MSE(pi1): 2.527e+00, MSE(pi2): 1.429e+00, MSE(pi3): 2.114e-01
Epoch 60, Train loss: 3.036e+07, Test loss: 3.620e+07, MSE(e): 3.031e+00, MSE(pi1): 2.510e+00, MSE(pi2): 1.399e+00, MSE(pi3): 2.121e-01
Epoch 70, Train l

In [28]:
# Parametros de entrenamiento
start_epoch = 80
n_epochs = 100

batch_size = 64
n_checkpoints = 10

second_lr = 1e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 80.
Epoch 80, Train loss: 2.857e+07, Test loss: 3.445e+07, MSE(e): 2.852e+00, MSE(pi1): 2.517e+00, MSE(pi2): 1.325e+00, MSE(pi3): 2.122e-01
Epoch 90, Train loss: 2.854e+07, Test loss: 3.441e+07, MSE(e): 2.849e+00, MSE(pi1): 2.516e+00, MSE(pi2): 1.323e+00, MSE(pi3): 2.122e-01

Training process finished after 100 epochs

